<h1 style="background-color:red;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐 הסבת שכבת חלקות (דו מימדיות) מבוטלות לבנק"ל מודרני 🌐

In [ ]:
print(' Retired Parcels:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import subprocess
import numpy as np
import pandas as pd
import geopandas as gpd
from configs import CNFG
import dask_geopandas as daskgpd

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('   Reading inputs')

<h1 style="text-align:right;font-size:125%">
נתוני שכבת חלקות היסטוריות

In [ ]:
#שדות אליפסה מתארים מה ביטל את החלקה

history_parcels_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'PARCEL', 'LEGAL_AREA', 'TALAR_NUM', 'TALAR_YEAR', 'geometry']

history_parcels = daskgpd.read_file(CNFG.CADSDE + 'PARCEL_HISTORY_HASAVA.gdb', layer = 'PARCEL_HISTORY_HASAVA', columns = history_parcels_cols, npartitions = 8).compute()

del history_parcels_cols

print('         input 1/8 read')

<h1 style="text-align:right;font-size:125%">
טבלת חלקות מבוטלות

In [ ]:
#שדות אליפסה מתארים מה ביטל את החלקה

cancel_parcel_cols = ['F_GUSH_NUM', 'F_GUSH_SUFFIX', 'F_PARCEL_NUM', 'TALAR_NUMBER', 'TALAR_YEAR']

cancel_parcel = pd.read_csv(CNFG.Oracle + 'CANCEL_PARCEL.csv', usecols = cancel_parcel_cols)

del cancel_parcel_cols

print('         input 2/8 read')

<h1 style="text-align:right;font-size:125%">
שכבת חלקות מבוטלות מרפליקציות יומיות

In [ ]:
#שדות אליפסה מתארים מה יצר את החלקה
replicated_parcels_cols = ['PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX', 'LEGALAREA', 'TALAR_NUMBER', 'TALAR_YEAR', 'geometry']

replicated_parcels = daskgpd.read_file(CNFG.Local + 'CanceldParcels.gdb',  layer = 'CanceledParcels', columns = replicated_parcels_cols, npartitions = 10).compute()

del replicated_parcels_cols

print('         input 3/8 read')

<h1 style="text-align:right;font-size:125%">
נתוני טבלת סדר פעולות

In [ ]:
sequence_cols = ['CPBUniqueID', 'ProcessType', 'BlockNumber', 'SubBlockNumber', 'FromParcelFinal', 'ToParcelFinal', 'ToBlockNumber', 'ToSubBlockNumber']

SequenceActions = pd.read_csv(CNFG.CSVPath + 'SequenceActions.csv', usecols = sequence_cols)

del sequence_cols

print('         input 4/8 read')

<h1 style="text-align:right;font-size:125%">
נתוני גושים מודרניים 

In [ ]:
blocks_cols = ['GlobalID', 'Name', 'LandType', 'IsTax']

Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = blocks_cols)
del blocks_cols

print('         input 5/8 read')

<h1 style="text-align:right;font-size:125%">
נתוני גבולות תהליכי ורישומי קדסטר

In [ ]:
Process_Records_cols = ['ProcessGUID', 'RecordGUID', 'ProcessName', 'ProcessType']

Process_Records = pd.read_csv(CNFG.CSVPath + 'ProcessesAndRecords.csv', usecols = Process_Records_cols)

del Process_Records_cols
print('         input 6/8 read')

<h1 style="text-align:right;font-size:125%">
רשימת מספרי חלקות רצף

In [ ]:
Parcels2D_List = pd.read_csv(CNFG.CSVPath + 'Parcels2DList.csv', usecols = ['Name'])

print('         input 7/8 read')

<h1 style="text-align:right;font-size:125%">
שכבת חלקות דו-ממדיות מודרנית

In [ ]:
Outliers_List = pd.read_csv(CNFG.Local + 'OutliersCanceledParcels2D.csv')['Name'].to_list()

print('         input 8/8 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('   Cleaning data')

<h1 style="text-align:right;font-size:100%">
סינון טבלת תהליכים ורישומי קדסטר לרישומי קדסטר בלבד (תהליכים מאושרים)

In [ ]:
Process_Records = Process_Records[Process_Records['RecordGUID'].notna()]

<h1 style="text-align:right;font-size:125%">
:עיבוד טבלת סדר פעולות לקבלת חלקות מתבטלות בלבד
<h1 style="text-align:right;font-size:100%">
סינון רישומי קדסטר לפי סוגי תהליכים מבטלים בלבד-
<h1 style="text-align:right;font-size:100%">
סינון טבלת סדר פעולות לפי סוגי תהליכים מבטלים בלבד-
<h1 style="text-align:right;font-size:100%">
סינון לפי הסרת חלקות שאין להן מספר סופי, כלומר שהתהליך הקדסטרי לא נרשם-
<h1 style="text-align:right;font-size:100%">
סינון לפי הסרת חלקות שמספרן הסופי זהה למספר הביסוס לפי התהליך (טעויות שקיימות בבסנ"ת נוכחי)-
<h1 style="text-align:right;font-size:100%">
הגדרת שדה של חלקה מבוטלת או כאשר חלקת ביסוס סופית לא זהה לחלקת סופית יוצאת-
<h1 style="text-align:right;font-size:100%">
שמירת שדות רלונטיים להמשך ושינוי השם שלהם-
<h1 style="text-align:right;font-size:100%">
הגדרת שדה לסוג תהליך מבטל כתצ"ר בלבד-

In [ ]:
CancelingProcessTypes = [1, 3, 11] # תצ"ר, פס"ד, תצ"ר לא מוסדר
CancelingProcessLayer = Process_Records[Process_Records['ProcessType'].isin(CancelingProcessTypes)]

SequenceActions_cancel = SequenceActions[SequenceActions['ProcessType'].isin(CancelingProcessTypes)]

SequenceActions_cancel = SequenceActions_cancel[(SequenceActions_cancel['FromParcelFinal'].notna()) & (SequenceActions_cancel['FromParcelFinal'] != 0)]
SequenceActions_cancel = SequenceActions_cancel[(SequenceActions_cancel['ToParcelFinal'].notna()) & (SequenceActions_cancel['ToParcelFinal'] != 0)]

SequenceActions_cancel[['BlockNumber', 'SubBlockNumber', 'FromParcelFinal']] = SequenceActions_cancel[['BlockNumber', 'SubBlockNumber', 'FromParcelFinal']].astype(int)
SequenceActions_cancel['Name'] = SequenceActions_cancel['FromParcelFinal'].astype(str) + '/' + SequenceActions_cancel['BlockNumber'].astype(str) + '/' + SequenceActions_cancel['SubBlockNumber'].astype(str)

SequenceActions_cancel = SequenceActions_cancel.merge(Process_Records[['ProcessGUID', 'ProcessName', 'RecordGUID']],
                                                      left_on = 'CPBUniqueID',
                                                      right_on = 'ProcessGUID',
                                                      how = 'left')\
                                                .drop(columns = 'ProcessGUID')\
                                                .rename(columns = {'RecordGUID' : 'RetiredByRecord',
                                                                   'ProcessType' : 'CancelProcessType'})

SequenceActions_cancel = SequenceActions_cancel[SequenceActions_cancel['ProcessName'].notna()]

SequenceActions_cancel = SequenceActions_cancel[['Name', 'CancelProcessType', 'RetiredByRecord', 'BlockNumber', 'SubBlockNumber']]

<h1 style="text-align:right;font-size:125%">
:עיבוד טבלת חלקות מבוטלות
<h1 style="text-align:right;font-size:100%">
שינוי שמות שדות-
<h1 style="text-align:right;font-size:100%">
הגדרת שדה של שם התכנית המבטלת-
<h1 style="text-align:right;font-size:100%">
מיזוג טבלת חלקות מבוטלות עם גבולות רישומי קדסטר מסוג תכניות מבטלות בלבד לקבלת מזהה תהליך מבטל-
<h1 style="text-align:right;font-size:100%">
הגדרת שדה של חלקה מבוטלת או כאשר חלקת ביסוס סופית לא זהה לחלקת סופית יוצאת-
<h1 style="text-align:right;font-size:100%">
שינוי שם השדה של התהליך המבטל-

In [ ]:
cancel_parcel.rename(columns = {'F_GUSH_NUM'    : 'BlockNumber',
                                'F_GUSH_SUFFIX' : 'SubBlockNumber',
                                'F_PARCEL_NUM'  : 'ParcelNumber'},
                     inplace = True)


cancel_parcel[['TALAR_NUMBER', 'TALAR_YEAR']] = cancel_parcel[['TALAR_NUMBER', 'TALAR_YEAR']].astype('Int64')
cancel_parcel['ProcessName'] = cancel_parcel['TALAR_NUMBER'].astype(str) + '/' + cancel_parcel['TALAR_YEAR'].astype(str)


cancel_parcel = cancel_parcel.merge(CancelingProcessLayer, on = 'ProcessName', how = 'left')\
                             .drop(columns = ['TALAR_NUMBER', 'TALAR_YEAR', 'ProcessName', 'ProcessGUID'])\
                             .rename(columns = {'RecordGUID' : 'RetiredByRecord',
                                                'ProcessType' : 'CancelProcessType'})


del CancelingProcessTypes

<h1 style="text-align:right;font-size:125%">
:עיבוד שכבת חלקות מבוטלות מרפליקציה יומית
<h1 style="text-align:right;font-size:100%">
שינוי שמות שדות-
<h1 style="text-align:right;font-size:100%">
הסרת חלקות שלא ידוע מה התכנית שביטלה אותן-
<h1 style="text-align:right;font-size:100%">
הגדרת שדה של שם התכנית המבטלת-
<h1 style="text-align:right;font-size:100%">
מיזוג טבלת חלקות מבוטלות עם גבולות רישומי קדסטר מסוג תכניות מבטלות בלבד לקבלת מזהה תהליך מבטל-
<h1 style="text-align:right;font-size:100%">
הגדרת שדה של חלקה מבוטלת או כאשר חלקת ביסוס סופית לא זהה לחלקת סופית יוצאת-
<h1 style="text-align:right;font-size:100%">
שינוי שם השדה של התהליך המבטל-

In [ ]:
replicated_parcels.rename(columns = {'PARCEL'      : 'ParcelNumber',
                                     'GUSH_NUM'    : 'BlockNumber',
                                     'GUSH_SUFFIX' : 'SubBlockNumber'}, inplace = True)

replicated_parcels = replicated_parcels[(~replicated_parcels['TALAR_NUMBER'].isna())
                                                          &
                                        (~replicated_parcels['TALAR_YEAR'].isna())]

replicated_parcels[['TALAR_NUMBER', 'TALAR_YEAR']] = replicated_parcels[['TALAR_NUMBER', 'TALAR_YEAR']].astype(int)
replicated_parcels['ProcessName'] = replicated_parcels['TALAR_NUMBER'].astype(str) + '/' + replicated_parcels['TALAR_YEAR'].astype(str)

replicated_parcels = replicated_parcels.merge(CancelingProcessLayer, on = 'ProcessName', how = 'left')\
                                       .drop(columns = ['TALAR_NUMBER', 'TALAR_YEAR', 'ProcessName', 'ProcessGUID'])\
                                       .rename(columns = {'RecordGUID' : 'CreatedByRecord',
                                                          'ProcessType' : 'CreateProcessType'})

<h1 style="text-align:right;font-size:100%">
עיבוד נתוני חלקות היסטוריות

In [ ]:
history_parcels.rename(columns = {'GUSH_NUM'     : 'BlockNumber',
                                  'GUSH_SUFFIX'  : 'SubBlockNumber',
                                  'PARCEL'       : 'ParcelNumber',
                                  'LEGAL_AREA'   : 'StatedArea'}, inplace = True)

history_parcels = history_parcels[(~history_parcels['TALAR_NUM'].isna())
                                                &
                                  (~history_parcels['TALAR_YEAR'].isna())]

history_parcels[['TALAR_NUM', 'TALAR_YEAR']] = history_parcels[['TALAR_NUM', 'TALAR_YEAR']].astype(int)
history_parcels['ProcessName'] = history_parcels['TALAR_NUM'].astype(str) + '/' + history_parcels['TALAR_YEAR'].astype(str)

history_parcels = history_parcels.merge(CancelingProcessLayer, on = 'ProcessName', how = 'left')\
                                 .drop(columns = ['TALAR_NUM', 'TALAR_YEAR', 'ProcessName', 'ProcessGUID'])\
                                 .rename(columns = {'RecordGUID' : 'RetiredByRecord',
                                                    'ProcessType' : 'CancelProcessType'})

In [ ]:
print('   Data conversion')

<h1 style="text-align:right;font-size:100%">
איחוד מקורות מידע של חלקות מבוטלות לטבלה אחת

In [ ]:
CancelParcels2D = pd.concat([cancel_parcel, replicated_parcels, history_parcels, SequenceActions_cancel], ignore_index = True)

CancelParcels2D['SubBlockNumber'] = CancelParcels2D['SubBlockNumber'].fillna(0)

CancelParcels2D = CancelParcels2D[(CancelParcels2D['ParcelNumber'] > 0) & (CancelParcels2D['ParcelNumber'].notna())]

del [cancel_parcel, replicated_parcels, history_parcels, SequenceActions_cancel]

<h1 style="text-align:right;font-size:100%">
הסרת חלקות מבוטלות אם הן מופיעות גם כחלקות רצף וקיבוץ הרשומות והסרת כפילויות

In [ ]:
CancelParcels2D[['BlockNumber', 'SubBlockNumber', 'ParcelNumber']] = CancelParcels2D[['BlockNumber', 'SubBlockNumber', 'ParcelNumber']].astype(int)
CancelParcels2D['Name'] = CancelParcels2D['ParcelNumber'].astype(str) + '/' + CancelParcels2D['BlockNumber'].astype(str) + '/' + CancelParcels2D['SubBlockNumber'].astype(str)

CancelParcels2D = CancelParcels2D[~CancelParcels2D['Name'].isin(Parcels2D_List['Name'])]

CancelParcels2D = CancelParcels2D.groupby(by = ['Name'], as_index = False).first()

del Parcels2D_List

<h1 style="text-align:right;font-size:100%">
מיפוי ערכי שדה סוג תהליך מבטל, ערכים ריקים יוגדרו כתהליך מבטל גנרי

In [ ]:
CancelProcessTypeMap = {1 : 1,       # תצ"ר
                        #2 : 4,       # תמ"ר
                        3 : 2,       # פס"ד
                        11 :3,       # תצ"ר לא מוסדר
                        16 : 16}
                        
CancelParcels2D['CancelProcessType'] = CancelParcels2D['CancelProcessType'].fillna(16)
CancelParcels2D['CancelProcessType'] = CancelParcels2D['CancelProcessType'].map(CancelProcessTypeMap)

CancelParcels2D['RetiredByRecord'] = np.where(CancelParcels2D['CancelProcessType'] == 16,
                                              CNFG.FabricatedCancelProcessGUID,
                                              CancelParcels2D['RetiredByRecord'])

<h1 style="text-align:right;font-size:100%">
מיזוג עם טבלת גושים לקבלת מספר מזהה לגוש, סוג מקרקעין ומאפיין שומא

In [ ]:
CadasterBlocks = Blocks[Blocks['IsTax'] == 0].rename(columns = {'GlobalID' : 'CadasterBlockUniqueID',
                                                                'Name'     : 'BlockName',
                                                                'LandType' : 'CadasterLandType',
                                                                'IsTax'    : 'IsTax_0'})

TaxBlocks = Blocks[Blocks['IsTax'] == 1].drop_duplicates('Name', keep = False)\
                                        .rename(columns = {'GlobalID' : 'TaxBlockUniqueID',
                                                           'Name'     : 'BlockName',
                                                           'LandType' : 'TaxLandType',
                                                           'IsTax'    : 'IsTax_1'})


CancelParcels2D[['BlockNumber', 'SubBlockNumber']] = CancelParcels2D[['BlockNumber', 'SubBlockNumber']].astype(int)
CancelParcels2D['BlockName'] = CancelParcels2D['BlockNumber'].astype(str) + '/' +  CancelParcels2D['SubBlockNumber'].astype(str) 


CancelParcels2D = CancelParcels2D.merge(CadasterBlocks, on = 'BlockName', how = 'left')\
                                 .merge(TaxBlocks, on = 'BlockName', how = 'left')\
                                 .drop(columns = 'BlockName')


CancelParcels2D['BlockUniqueID'] = CancelParcels2D['CadasterBlockUniqueID'].fillna(CancelParcels2D['TaxBlockUniqueID'])

CancelParcels2D['LandType'] = CancelParcels2D['CadasterLandType'].fillna(CancelParcels2D['TaxLandType'])

CancelParcels2D['IsTax'] = CancelParcels2D['IsTax_0'].fillna(CancelParcels2D['IsTax_1'])


CancelParcels2D.drop(columns = ['CadasterBlockUniqueID', 'TaxBlockUniqueID', 'CadasterLandType', 'TaxLandType', 'IsTax_0', 'IsTax_1'], inplace = True)


del [Blocks, CadasterBlocks, TaxBlocks]

<h1 style="text-align:right;font-size:125%">
עיבוד לטבלת סדר פעולות לקבלת מידע של תהליכים יוצרים

In [ ]:
SequenceActions_created = SequenceActions[(SequenceActions['ToParcelFinal'].notna())
                                                          &
                                          (SequenceActions['ToParcelFinal'] != 0)
                                                          &
                                          (~SequenceActions['ProcessType'].isin([4, 6]))] #תת"ג, 97ב


SequenceActions_created = SequenceActions_created.merge(Process_Records[['ProcessGUID', 'RecordGUID']],
                                                        left_on = 'CPBUniqueID', right_on = 'ProcessGUID',
                                                        how = 'inner')\
                                                 .drop(columns = ['ProcessGUID', 'CPBUniqueID'])\
                                                 .dropna(subset = 'RecordGUID')\
                                                 .rename(columns = {'RecordGUID' : 'CreatedByRecord', 'ProcessType' : 'CreateProcessType'})


SequenceActions_created['ToParcelFinal'] = SequenceActions_created['ToParcelFinal'].astype(int)

SequenceActions_created['ToBlockNumber'] = SequenceActions_created['ToBlockNumber'].replace(0, None)
SequenceActions_created['ToBlockNumber'] = SequenceActions_created['ToBlockNumber'].fillna(SequenceActions_created['BlockNumber'])

SequenceActions_created['ToSubBlockNumber'] = SequenceActions_created['ToSubBlockNumber'].replace(0, None)
SequenceActions_created['ToSubBlockNumber'] = SequenceActions_created['ToSubBlockNumber'].fillna(SequenceActions_created['SubBlockNumber'])
SequenceActions_created['ToSubBlockNumber'] = SequenceActions_created['ToSubBlockNumber'].fillna(0)

SequenceActions_created[['ToBlockNumber', 'ToSubBlockNumber']] = SequenceActions_created[['ToBlockNumber', 'ToSubBlockNumber']].astype(int)


SequenceActions_created['Name'] = SequenceActions_created['ToParcelFinal'].astype(str) + '/' +\
                                  SequenceActions_created['ToBlockNumber'].astype(str) + '/' +\
                                  SequenceActions_created['ToSubBlockNumber'].astype(str)

SequenceActions_created = SequenceActions_created[SequenceActions_created['Name'].isin(CancelParcels2D['Name'])][['Name', 'CreatedByRecord', 'CreateProcessType']]

SequenceActions_created = SequenceActions_created.drop_duplicates(keep = 'first')

CancelParcels2D = CancelParcels2D.merge(SequenceActions_created, on = 'Name', how = 'left')

CancelParcels2D = CancelParcels2D.drop_duplicates(keep = 'first')

CancelParcels2D['CreateProcessType'] = CancelParcels2D['CreateProcessType_x'].fillna(CancelParcels2D['CreateProcessType_y'])
CancelParcels2D['CreatedByRecord'] = CancelParcels2D['CreatedByRecord_x'].fillna(CancelParcels2D['CreatedByRecord_y'])
CancelParcels2D.drop(columns = ['CreateProcessType_x', 'CreateProcessType_y', 'CreatedByRecord_x', 'CreatedByRecord_y'], inplace = True)

CreateProcessTypeMap = {1  : 1,      # תצ"ר
                        3  : 2,      # פס"ד
                        11 : 4,      # תצ"ר לא מוסדר
                        16 : 16}      

CancelParcels2D['CreateProcessType'] = CancelParcels2D['CreateProcessType'].fillna(16)
CancelParcels2D['CreateProcessType'] = CancelParcels2D['CreateProcessType'].map(CreateProcessTypeMap)

CancelParcels2D['CreatedByRecord'] = CancelParcels2D['CreatedByRecord'].fillna(CNFG.FabricatedCreateProcessGUID)

del [SequenceActions, SequenceActions_created, CreateProcessTypeMap, Process_Records]

<h1 style="text-align:right;font-size:100%">
הגדרת שדות מספרים שלמים

In [ ]:
int_cols = ['ParcelNumber','BlockNumber','SubBlockNumber','CancelProcessType','CreateProcessType', 'LandType']

CancelParcels2D[int_cols] = CancelParcels2D[int_cols].astype('Int64')

<h1 style="text-align:right;font-size:100%">
הסרת גיאומטריה לחלקות מבוטלות שמחוץ לגבולות ארץ ישראל

In [ ]:
CancelParcels2D.loc[CancelParcels2D['Name'].isin(Outliers_List), 'geometry'] = None
del Outliers_List

In [ ]:
CancelParcels2D = CancelParcels2D.groupby(by = 'Name', as_index = False).last()

<h1 style="text-align:right;font-size:100%">
איכלוס שדה סוג חלקה

In [ ]:
CancelParcels2D['ParcelType'] = 2 # סופית

<h1 style="text-align:right;font-size:100%">
איכלוס שדה יחדית שטח

In [ ]:
CancelParcels2D['StatedAreaUnit'] = 109404 #מטר מרובע

<h1 style="text-align:right;font-size:100%">
הגדרת התוצאות כטבלה גיאוגרפית

In [ ]:
CancelParcels2D = gpd.GeoDataFrame(CancelParcels2D, crs = 2039, geometry = 'geometry')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

In [ ]:
print('   Exporting results')

In [ ]:
CancelParcels2D.to_file(CNFG.Results, layer = 'CancelParcels2D', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')

In [ ]:
print('   Appending results to product')

In [ ]:
subprocess.call([CNFG.BatchPath + "Parcels2D(retired)-Save as Feature Class.bat"]);